In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("C:/Users/wonny/Downloads/final2.csv")

In [3]:
data

,Year,LocationAbbr,LocationDesc,Geolocation,Disease_Type,Data_Value_Type,Data_Value,Break_Out_Category,Break_Out_Details,Life_Expectancy
0,2000,AL,Alabama,POINT (-86.63186076 32.84057112),0,1,508.2,Overall,Overall,74.364763
1,2000,AL,Alabama,POINT (-86.63186076 32.84057112),0,0,525.6,Overall,Overall,74.364763
2,2000,AL,Alabama,POINT (-86.63186076 32.84057112),0,0,514.9,Gender,Male,NaN
3,2000,AL,Alabama,POINT (-86.63186076 32.84057112),0,1,550.4,Gender,Male,NaN
4,2000,AL,Alabama,POINT (-86.63186076 32.84057112),0,1,471.7,Gender,Female,NaN
...,...,...,...,...,...,...,...,...,...,...
85675,2020,WY,Wyoming,POINT (-108.1098304 43.23554134),7,1,10.2,Gender,Female,NaN
85676,2020,WY,Wyoming,POINT (-108.1098304 43.23554134),7,0,NaN,Age,18-24,NaN
85677,2020,WY,Wyoming,POINT (-108.1098304 43.23554134),7,0,NaN,Age,25-44,NaN
85678,2020,WY,Wyoming,POINT (-108.1098304 43.23554134),7,0,14.2,Age,45-64,NaN


In [9]:
# 필터링: Data_Value_Type이 0인 행만 선택
filtered_data = data[data["Data_Value_Type"] == 0]

# 필요한 컬럼 선택: 'Year', 'LocationDesc', 'Disease_Type', 'Break_Out_Category', 'Break_Out_Details', 'Data_Value'
filtered_data = filtered_data[
    [
        "Year",
        "LocationDesc",
        "Disease_Type",
        "Break_Out_Category",
        "Break_Out_Details",
        "Data_Value",
    ]
]

# Creating a combined column for Break Out Category and Details
filtered_data["CategoryDetail"] = (
    filtered_data["Break_Out_Category"] + "_" + filtered_data["Break_Out_Details"]
)

# Pivoting the data
pivot_table = filtered_data.pivot_table(
    values="Data_Value",
    index=["Year", "LocationDesc", "Disease_Type"],
    columns=["CategoryDetail"],
    aggfunc="first",
)

# Resetting index to make 'Year', 'LocationDesc', and 'Disease_Type' as columns
pivot_table.reset_index(inplace=True)

# Adding the 'Life_Expectancy' column from the original data
pivot_table["Life_Expectancy"] = data["Life_Expectancy"]

pivot_table

CategoryDetail,Year,LocationDesc,Disease_Type,Age_18-24,Age_25-44,Age_45-64,Age_65+,Gender_Female,Gender_Male,Overall_Overall,Life_Expectancy
0,2000,Alabama,0,5.4,38.2,273.4,2442.8,535.3,514.9,525.6,74.364763
1,2000,Alabama,1,4.5,30.2,220.9,1844.7,394.7,410.0,402.0,74.364763
2,2000,Alabama,2,NaN,7.6,86.1,482.3,102.8,125.4,113.5,NaN
3,2000,Alabama,3,NaN,13.0,133.5,1024.1,206.1,245.5,224.7,NaN
4,2000,Alabama,4,NaN,2.0,16.2,328.2,73.1,52.1,63.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...
8563,2020,Wyoming,3,NaN,NaN,91.3,479.9,101.0,179.9,141.1,NaN
8564,2020,Wyoming,4,NaN,NaN,NaN,130.1,31.2,33.3,32.3,NaN
8565,2020,Wyoming,5,NaN,NaN,18.4,184.1,53.4,45.1,49.2,NaN
8566,2020,Wyoming,6,NaN,NaN,NaN,114.7,32.2,23.2,27.6,NaN


In [12]:
# Creating a function to map the year to the nearest life expectancy year (2000, 2005, 2010, 2015)
def map_to_life_expectancy_year(year):
    if year < 2005:
        return 2000
    elif year < 2010:
        return 2005
    elif year < 2015:
        return 2010
    else:
        return 2015


# Applying the function to the 'Year' column
filtered_data["MappedYear"] = filtered_data["Year"].apply(map_to_life_expectancy_year)

# Merging the original data to get the life expectancy values for the mapped years
merged_data = (
    pd.merge(
        filtered_data,
        data[["Year", "LocationDesc", "Life_Expectancy"]].drop_duplicates(),
        left_on=["MappedYear", "LocationDesc"],
        right_on=["Year", "LocationDesc"],
        how="left",
    )
    .drop("Year_y", axis=1)
    .rename(columns={"Year_x": "Year"})
)

# Pivoting the merged data
pivot_table = merged_data.pivot_table(
    values="Data_Value",
    index=["Year", "LocationDesc", "Disease_Type", "Life_Expectancy"],
    columns=["CategoryDetail"],
    aggfunc="first",
).reset_index()

pivot_table.head(50)

CategoryDetail,Year,LocationDesc,Disease_Type,Life_Expectancy,Age_18-24,Age_25-44,Age_45-64,Age_65+,Gender_Female,Gender_Male,Overall_Overall
0,2000,Alabama,0,74.364763,5.4,38.2,273.4,2442.8,535.3,514.9,525.6
1,2000,Alabama,1,74.364763,4.5,30.2,220.9,1844.7,394.7,410.0,402.0
2,2000,Alabama,2,74.364763,NaN,7.6,86.1,482.3,102.8,125.4,113.5
3,2000,Alabama,3,74.364763,NaN,13.0,133.5,1024.1,206.1,245.5,224.7
4,2000,Alabama,4,74.364763,NaN,2.0,16.2,328.2,73.1,52.1,63.2
5,2000,Alabama,5,74.364763,NaN,6.0,38.4,465.8,112.1,77.0,95.5
6,2000,Alabama,6,74.364763,NaN,NaN,18.3,332.0,78.0,48.5,64.1
7,2000,Alabama,7,74.364763,NaN,4.2,17.8,81.1,23.0,19.6,21.4
8,2000,Alaska,0,76.391839,NaN,17.7,141.0,1612.1,176.7,196.7,187.1
9,2000,Alaska,1,76.391839,NaN,14.3,111.8,1155.1,122.1,153.5,138.4


In [13]:
# Removing the 'CategoryDetail' column
pivot_table_cleaned = pivot_table.drop("CategoryDetail", axis=1, errors="ignore")

# Setting the path for the new CSV file
output_file_path = "pivoted_data.csv"

# Saving the dataframe to a CSV file
pivot_table_cleaned.to_csv(output_file_path, index=False)

output_file_path

'pivoted_data.csv'

In [2]:
import pandas as pd

df = pd.read_csv("data/pivoted_data.csv")

year = 2020
state_name = "Texas"
disease_type = 0
field_name = "Age_65+"

# 데이터 필터링
filtered_df = df[(df["Year"] == year) & (df["Disease_Type"] == disease_type)]
if state_name:
    filtered_df = filtered_df[filtered_df["LocationDesc"] == state_name]

# 필터링된 데이터 프레임 검사
if not filtered_df.empty and field_name in filtered_df.columns:
    value = filtered_df[field_name].iloc[0]
    if pd.notna(value):  # 값이 NaN이 아닌지 확인
        print("Value found:", value)
    else:
        print("Value is NaN")
else:
    if filtered_df.empty:
        print("No data found for the given filters.")
    if field_name not in filtered_df.columns:
        print(f"Column {field_name} does not exist in the DataFrame.")

Value found: 1315.2


In [3]:
print(df.columns.tolist())

['Year', 'LocationDesc', 'Disease_Type', 'Life_Expectancy', 'Age_18-24', 'Age_25-44', 'Age_45-64', 'Age_65+', 'Gender_Female', 'Gender_Male', 'Overall_Overall']
